In [1]:
import pandas as pd
import csv
import pandas as pd
import glob
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy.fft import fft, fftfreq
from scipy.signal import firwin, lfilter, butter
# import asrpy
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
# import cv2

2022-06-11 18:45:34.442551: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/neaf-2070/.mujoco/mujoco200_linux/bin:/usr/local/cuda-10.2/lib64:/usr/local/cuda/lib64:/home/neaf-2070/.mujoco/mujoco200_linux/bin:/usr/local/cuda-10.2/lib64:/usr/local/cuda/lib64:
2022-06-11 18:45:34.442571: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
fs = 500

In [3]:
data_pf = pd.read_csv("combine_all_df.csv")

In [4]:
data_l = ["HandStart", "FirstDigitTouch", "BothStartLoadPhase", "LiftOff", "Replace", "BothReleased"]
data_f = ["FC1", "FC2", "FC5", "FC6"]
data_m = ["C3", "C4", "T7", "T8"]
data_b = ["TP9", "TP10", "CP1", "CP2", "CP5", "CP6"]
data_u = ["Unnamed: 0", "Unnamed: 0_x", "id"]

In [5]:
def datacut(data, chi, chf):
    c_sum = 0
    step = 0
    dataset = []
    labels = []
    for i in range(1,len(data)):
        if (data.iloc[i,-6:] - data.iloc[i-1,-6:]).sum() == 0:
            c_sum = c_sum + 1
        else:
            c_sum = 0
        if c_sum > 100:
            step += 1
        if c_sum == 100 or step == 10:
            step = 0
            dataset.append(data.iloc[i-100:i,chi:chf])
            labels.append(data.iloc[i-50,-6:])
    return dataset,labels


def bandpass(input_x, lf, hf, fs):
    nyq = 0.5 * fs
    low = lf / nyq
    high = hf / nyq
    b, a = butter(5, [low, high], btype='band')
    output_x = lfilter(b, a, input_x)
    return output_x

def preprocessing(data_list ,fs,chf,chi):
    data_fft_list = []
    data_Beta_list = []
    data_Bandpass_list = []
    for j in range(0,len(data_list)):
        aa = data_list[j]
        a = np.array(aa.iloc[:,0])
        data_Bandpass = bandpass(a,1,50,fs)
        fft = np.fft.fft(a)
        data_Beta = abs(fft[12:30])/np.linalg.norm(abs(fft[12:30]))
        data_fft = abs(fft[1:50])/np.linalg.norm(abs(fft[1:50]))
        for i in range(1,chf-chi):
            a = np.array(aa.iloc[:,i])
            a = bandpass(a,1,50,fs)
            fft = np.fft.fft(a)
            Beta = abs(fft[12:30])/np.linalg.norm(abs(fft[12:30]))
            fft = abs(fft[1:50])/np.linalg.norm(abs(fft[1:50]))
            
            data_fft = np.vstack([data_fft,fft])
            data_Beta = np.vstack([data_Beta,Beta])
            data_Bandpass = np.vstack([data_Bandpass,a])

        
        data_fft = data_fft.T
        data_Beta = data_Beta.T
        data_Bandpass = data_Bandpass.T
        
        scaler = StandardScaler().fit(data_fft)
        data_fft = scaler.transform(data_fft)
        scaler = StandardScaler().fit(data_Beta)
        data_Beta = scaler.transform(data_Beta)
        scaler = StandardScaler().fit(data_Bandpass)
        data_Bandpass = scaler.transform(data_Bandpass)
        
        data_fft_list.append(data_fft)
        data_Beta_list.append(data_Beta)
        data_Bandpass_list.append(data_Bandpass)
    return data_fft_list, data_Beta_list, data_Bandpass_list


def extracts(data,fs,chi,chf):
    dataset, labels           = datacut(data,chi,chf)
    dataset_fft_list, dataset_Beta_list, dataset_Bandpass_list= preprocessing(dataset, fs, chf, chi)

    
    return [dataset_fft_list, dataset_Beta_list, dataset_Bandpass_list, labels]





In [ ]:
[data_fft_list, data_Beta_list, data_Bandpass_list, labels] = extracts(data_pf,fs,3,35)



In [ ]:
len(data_fft_list)

In [ ]:
data_fft_list[0].shape

In [ ]:
data_traget = data_pf[data_u+ data_f+ data_m+ data_b+ data_l]

In [ ]:
[data_traget_fft_list, data_traget_Beta_list, data_traget_Bandpass_list, labels_x] = extracts(data_traget,fs,3,17)


In [ ]:
import pickle

with open('dataset_fft.pickle', 'wb') as f:
    pickle.dump(data_fft_list, f)
with open('dataset_Beta.pickle', 'wb') as f:
    pickle.dump(data_Beta_list, f)
with open('dataset_Bandpass.pickle', 'wb') as f:
    pickle.dump(data_Bandpass_list, f)

with open('dataset_fft_tragards.pickle', 'wb') as f:
    pickle.dump(data_traget_fft_list, f)
with open('dataset_Beta_tragards.pickle', 'wb') as f:
    pickle.dump(data_traget_Beta_list, f)
with open('dataset_Bandpass_tragards.pickle', 'wb') as f:
    pickle.dump(data_traget_Bandpass_list, f)

with open('labels.pickle', 'wb') as f:
    pickle.dump(labels, f)